In [1]:
from tensorflow import keras
import numpy as np

f_mnist = keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = f_mnist.load_data()

x_train_full = np.reshape(x_train_full, (x_train_full.shape[0], x_train_full.shape[1] * x_train_full.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))

print(f"Training dataset: {x_train_full.shape}, {x_train_full.dtype}")
print(f"Testing dataset: {x_test.shape}, {x_test.dtype}")

x_train = x_train_full / 255.0
y_train = y_train_full

2024-03-18 11:14:18.827077: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 11:14:18.866924: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 11:14:19.085514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 11:14:19.085562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 11:14:19.128454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Training dataset: (60000, 784), uint8
Testing dataset: (10000, 784), uint8


In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [3]:
from sklearn.model_selection import train_test_split

# Розділіть навчальний набір на два піднабори
x_train_partial, x_semi_auto, y_train_partial, y_semi_auto = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
y_semi_auto = np.full_like(y_semi_auto, fill_value=-1)

print(f"Partial Training dataset: {x_train_partial.shape}, {y_train_partial.shape}")
print(f"Semi-Auto Training dataset: {x_semi_auto.shape}, {y_semi_auto.shape}")

Partial Training dataset: (48000, 784), (48000,)
Semi-Auto Training dataset: (12000, 784), (12000,)


In [4]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC

base_classifier = SVC(kernel='rbf', gamma=0.5, C=0.1, probability=True)
classifier = SelfTrainingClassifier(base_classifier)
classifier.fit(x_train_partial, y_train_partial, x_semi_auto)

y_pred = classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

TypeError: SelfTrainingClassifier.fit() takes 3 positional arguments but 4 were given